## Image classification via KNN

In [1]:
import argparse
from imutils import paths
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [2]:
from utilities.preprocessing import SimplePreprocessor
from utilities.datasets import SimpleDatasetLoader


In [3]:
basedir='.'

In [4]:
#Get list of image paths which specifying the file paths to the images in our dataset residing on disk
image_paths = list(paths.list_images(basedir))
#print(image_paths[0]) #PRINT THE PATH NAME OF FIRST IMAGE

In [5]:
#Initialize SimplePreprocessor and SimpleDatasetLoader and load data and labels
print('[INFO]: Images loading....')
#load an image and resize to a fixed size
sp = SimplePreprocessor(32, 32) #resize to 32 x 32 pixel
sdl = SimpleDatasetLoader(preprocessors=[sp])
(data, labels) = sdl.load(image_paths, verbose=500)

[INFO]: Images loading....
[INFO]: Processed 500/3000
[INFO]: Processed 1000/3000
[INFO]: Processed 1500/3000
[INFO]: Processed 2000/3000
[INFO]: Processed 2500/3000
[INFO]: Processed 3000/3000


In [6]:
print('data shape:', data.shape) #3000 images of 32 X 32, 3 color channels
print(labels.shape)

data shape: (3000, 32, 32, 3)
(3000,)


Flatten our images from a 3D representation to a single list of pixel intensities.eshape from (3000, 32, 32, 3) to (3000, 32 x32x3=3072)  

this means each image is represented by 3072 integers

In [7]:
data = data.reshape((data.shape[0], 3072))
data.shape


(3000, 3072)

In [8]:
# Print information about memory consumption
convertFromByteToMB=1./ (1024*1000.0)
print('[INFO]: Features Matrix: {:.1f}MB'.format(float(data.nbytes*convertFromByteToMB)))


[INFO]: Features Matrix: 9.0MB


In [9]:
#our feature vectors are raw RGB pixel intensities. knn relies on the distance between feature vectors

#Encode labels as integers
le = LabelEncoder() #example ; cat=0, dog=1, panda=2, etc
labels = le.fit_transform(labels)


In [10]:
# Split data into training (75%) and testing (25%) data
(train_x, test_x, train_y, test_y) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [18]:
# Train and evaluate the k-NN classifier on the raw pixel intensities
print('[INFO]: Classification starting....')
num_neigh=1
num_jobs=-1
model = KNeighborsClassifier(n_neighbors=num_neigh,n_jobs=num_jobs)

[INFO]: Classification starting....


In [19]:
model.fit(train_x, train_y)
print(classification_report(test_y, model.predict(test_x),target_names=le.classes_))

             precision    recall  f1-score   support

       cats       0.40      0.54      0.46       239
       dogs       0.40      0.50      0.44       249
      panda       0.75      0.32      0.45       262

avg / total       0.52      0.45      0.45       750



the base model with random guessing would by 1/3, KNN with out any 'learning' beats the naive model